# GRT Parallel Processing

In this notebook we will test the parallel processing for GRT

In [36]:
from gravray import *
from gravray.util import *
from gravray.sampling import *
from gravray.spice import *
from gravray.orbit import *
from gravray.stats import *

from tqdm import tqdm
import pandas as pd
import multiprocessing as mp
import psutil
from itertools import product as cartesian

Spice.loadKernels()
NP=mp.cpu_count()
print("Number of processors: ",NP)

Number of processors:  4


In [23]:
#import ray
#np=psutil.cpu_count(logical=False)
#ray.init(num_cpus=2)

## Common data

In [24]:
body="EARTH"
earth=Body(body)

## Parallel function

In [99]:
def rayProcessing(initial):
    t=initial[0]
    site=initial[1]
    direction=initial[2]
    #print(f"Processing ray at {t}")
    ray=GrtRay(site,direction[0],direction[1],direction[2])
    ray.updateRay(t)
    ray.propagateRay(t)
    raydf=ray.packRay()
    raydf["detJ"]=ray.calcJacobianDeterminant()
    return raydf

## Input data

In [84]:
ts=[Spice.str2t("02/15/2013 03:20:34 UTC")]
siteprops=[[61.1*Angle.Deg,54.8*Angle.Deg,23.3*Const.km]]
sites=[]
for siteprop in siteprops:
    sites+=[Location(earth,siteprop[0],siteprop[1],siteprop[2])]
directions=[[101.1*Angle.Deg,15.9*Angle.Deg,-18.6*Const.km/Const.s]]

#List of conditions
initials=list(cartesian(*[ts,sites,directions]))

In [85]:
#Test
for initial in initials:
    r=rayProcessing(initial)
r    

Processing ray at 414170434.0


,et,lon,lat,alt,A,h,v,ximp,yimp,zimp,...,vzhel,q,e,i,W,w,M,a,n,detJ
0,414170434.0,61.1,54.8,23300.0,101.1,15.9,-18600.0,-1.232675e+11,8.134799e+10,-3.528140e+06,...,-2274.5822,0.738582,0.549668,4.041579,326.572556,106.86342,21.323547,1.640082,9.479146e-08,-4.509452e-13


In [151]:
#Times
print("Preparing times...")
tini=Spice.str2t("02/15/2013 03:20:34")
tend=tini+Const.Year
ts=np.linspace(tini,tend,100)

#Sites
print("Preparing sites...")
elTime(0)
H=23.3*Const.km
siteprops=[[61.1*Angle.Deg,54.8*Angle.Deg,H]]
sites=[]
for siteprop in siteprops:
    sites+=[Location(earth,siteprop[0],siteprop[1],siteprop[2])]
elTime()

#Directions
print("Preparing directions...")
elTime(0)
points=Sample(1000)
points.genUnitHemisphere()
points.pp[:,1]-=np.pi/2
speeds=np.linspace(11.2,72.0,50)*Const.km/Const.s
directions=list(cartesian(*[points.pp[:,1:],speeds]))
elTime()

#Initial conditions
print("Preparing initial conditions...")
elTime(0)
initials=list(cartesian(*[ts,sites,directions]))
elTime()

Ninitials=len(initials)
print(f"Number of initial conditions: {len(initials)} = {len(ts)}*{len(sites)}*{len(directions)}")

Preparing times...
Preparing sites...
Elapsed time since last call: 1.94788 ms
Preparing directions...
Elapsed time since last call: 50.9129 ms
Preparing initial conditions...
Elapsed time since last call: 3.21778 s
Number of initial conditions: 5000000 = 100*1*50000


In [ ]:
points.pp[:,1]

## Parallel processing

In [114]:
%timeit -n 100 rayProcessing(initials[0])

100 loops, best of 3: 28.2 ms per loop


In [143]:
raydfs=pd.DataFrame()
def joinResults(raydf):
    global raydfs
    raydfs=pd.concat((raydfs,raydf))

In [ ]:
pool=mp.Pool(NP)
elTime(0)
#R=[pool.apply_async(rayProcessing,args=(initial,),callback=joinResults) for initial in initials]
for initial in tqdm(initials):
    pool.apply_async(rayProcessing,args=(initial,),callback=joinResults)
pool.close()
pool.join()
elTime()

100%|██████████| 5000000/5000000 [06:09<00:00, 13519.77it/s]


In [ ]:
print("Number of results:",len(raydfs))

In [ ]:
raydfs.to_csv("rays_parallel.csv")